# Purpose
This notebook creates a baseline model as a point of reference for future model iterations. This model predicts the total number of significant strikes landed given two fighters average significant strikes landed in all of their previous bouts.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import PoissonRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

### Import and split

In [3]:
data = pd.read_csv('../../data/modelling_data/fsm_data.csv', index_col=0)
data.head()

,ASSL_0,ASSL_1,TSSL_bout
0,10.333333,16.333333,54
1,20.000000,32.250000,31
2,38.600000,20.230769,19
3,32.600000,23.000000,60
4,20.200000,37.833333,148


In [4]:
X = data.drop('TSSL_bout', axis=1)
y = data.TSSL_bout

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)

### Preprocessing

In [6]:
ss = StandardScaler()
X_train_ss = ss.fit_transform(X_train)
X_test_ss = ss.transform(X_test)

### Modelling

In [7]:
pr = PoissonRegressor()
cross_val_score(pr, X_train_ss, y_train, scoring='r2')

array([0.04569839, 0.11870429, 0.15341573, 0.13209458, 0.13813617])

In [8]:
pr.fit(X_train_ss, y_train)

PoissonRegressor()

### Evaluation

In [9]:
y_hat = pr.predict(X_test_ss)

In [10]:
r2_score(y_test, y_hat)

0.11153337471858615

#### Poisson Deviance

In [11]:
pr.score(X_test_ss, y_test)

0.12224935153515593

In [12]:
results = pd.DataFrame({'model_predictions': list(y_hat), 'actual_results': list(y_test)})
results.iloc[:20]

,model_predictions,actual_results
0,66.382847,50
1,47.522719,20
2,60.851422,55
3,67.088840,32
4,58.366670,34
5,52.497366,39
6,94.719385,35
7,63.939038,56
8,53.213855,15
9,76.572234,95


In [13]:
def compare_within_window(row):
    """
    givern a row for the dataframe above, returns True if the prediction is
    within 5 strikes of the actual result.
    """
    pred = row['model_predictions']
    true = row['actual_results']
    return pred>=(true-5) and pred<=(true+5)

In [14]:
accuracy_within_window = results.apply(compare_within_window, axis=1)

In [15]:
accuracy_within_window.mean()

0.07128712871287128

The model is prediciton almost half of the fights within a 5 strike window. When it does get it right, what is it guessing?